In [2]:
# jupyter notebook --NotebookApp.kernel_spec_manager_class='environment_kernels.EnvironmentKernelSpecManager'
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

import math
import sys, traceback
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

In [30]:
# data generating script
def place_cube(matrix, origin, r):
	#dim = matrix.shape()
	matrix[
	origin[0]-r:origin[0]+r,
	origin[1]-r:origin[1]+r,
	origin[2]-r:origin[2]+r,
	] = 1
	return matrix

def dist(x, y):
	return math.sqrt(
		(x[0]-y[0])**2+
		(x[1]-y[1])**2+
		(x[2]-y[2])**2
		)

def place_sphere(matrix, origin, r):
	for index, x in np.ndenumerate(matrix):
	    if dist(index, origin) <= r:
	    	matrix[index] = 1
	return matrix

def random_shapes(m_size,num):
    shapes = np.zeros((num,m_size,m_size,m_size))
    shape_types = np.random.randint(low = 0,high = 2,size= num)
    
    for i in range(num):
        if(i%(num/20) == 0):print(i/num)
        r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
        org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
        if shape_types[i] == 0:
            shapes[i,:,:,:] = place_cube(shapes[i,:,:,:],org,r)
        elif shape_types[i] == 1:
            shapes[i,:,:,:] = place_sphere(shapes[i,:,:,:],org,r)
            
    return shapes, shape_types
        

def matching_shapes(m_size, num):
	cubes = np.zeros((m_size,m_size,m_size,num))
	spheres = np.zeros((m_size,m_size,m_size,num))
	for i in range(num):
		r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
		org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
		cubes[:,:,:,i] = place_cube(cubes[:,:,:,i],org,r)
		spheres[:,:,:,i] = place_sphere(spheres[:,:,:,i],org,r)
	return cubes, spheres

In [70]:
FC_SIZE = 1024
DTYPE = tf.float32


def _weight_variable(name, shape):
    return tf.get_variable(name, shape, DTYPE, tf.truncated_normal_initializer(stddev=0.1))


def _bias_variable(name, shape):
    return tf.get_variable(name, shape, DTYPE, tf.constant_initializer(0.1, dtype=DTYPE))

def inference(features):
     # Input Layer
    # input_layer = tf.reshape(features, [-1, 28, 28, 1])
    prev_layer = features
    num_props = 1
    num_classes = 2
    
    in_filters = num_props
    with tf.variable_scope('conv1') as scope:
        out_filters = 16
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope.name)

        prev_layer = conv1
        in_filters = out_filters

    pool1 = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')
    norm1 = pool1  # tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta = 0.75, name='norm1')

    prev_layer = norm1

    with tf.variable_scope('conv2') as scope:
        out_filters = 32
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope.name)

        prev_layer = conv2
        in_filters = out_filters

    # normalize prev_layer here
    prev_layer = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

    with tf.variable_scope('conv3_1') as scope:
        out_filters = 64
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    with tf.variable_scope('conv3_2') as scope:
        out_filters = 64
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    with tf.variable_scope('conv3_3') as scope:
        out_filters = 32
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    # normalize prev_layer here
    prev_layer = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')


    with tf.variable_scope('local3') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        prev_layer_flat = tf.reshape(prev_layer, [-1, dim])
        weights = _weight_variable('weights', [dim, FC_SIZE])
        biases = _bias_variable('biases', [FC_SIZE])
        local3 = tf.nn.relu(tf.matmul(prev_layer_flat, weights) + biases, name=scope.name)

    prev_layer = local3

    with tf.variable_scope('local4') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        prev_layer_flat = tf.reshape(prev_layer, [-1, dim])
        weights = _weight_variable('weights', [dim, FC_SIZE])
        biases = _bias_variable('biases', [FC_SIZE])
        local4 = tf.nn.relu(tf.matmul(prev_layer_flat, weights) + biases, name=scope.name)

    prev_layer = local4

    with tf.variable_scope('softmax_linear') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        weights = _weight_variable('weights', [dim, num_classes])
        biases = _bias_variable('biases', [num_classes])
        softmax_linear = tf.add(tf.matmul(prev_layer, weights), biases, name=scope.name)

    return softmax_linear

def cnn_model_fn(features, labels, mode):
  #"""Model function for CNN."""
  # Input Layer

    #input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Logits Layer
    logits = inference(features)

    loss = None
    train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.001,
            optimizer="SGD")

  # Generate Predictions
    predictions = {
        "classes": tf.argmax(
          input=logits, axis=1),
        "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

  # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
        mode=mode, predictions=predictions, loss=loss, train_op=train_op)


In [71]:
def add_channel(data):
    return np.expand_dims(data, axis = data.ndim)

In [75]:
def main(unused_argv):
    # Load training and eval data
#     mnist = learn.datasets.load_dataset("mnist")
#     train_data = mnist.train.images # Returns np.array
#     train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
#     eval_data = mnist.test.images # Returns np.array
#     eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

    tf.logging.info('loading data...')
    train_data = add_channel(np.load('rand_shapes_data_small.npy'))
    train_labels = add_channel(np.load('rand_shapes_labels_small.npy'))
    
    eval_data = add_channel(np.load('rand_shapes_data_test.npy'))
    eval_labels = add_channel(np.load('rand_shapes_labels_test.npy'))
    tf.logging.info('DONE')
    
    shapes_classifier = learn.SKCompat(learn.Estimator(
      model_fn=cnn_model_fn, model_dir="shapes_convnet_model"))

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

        # Train the model
    shapes_classifier.fit(
        x=train_data,
        y=train_labels,
        batch_size=10,
        steps=100,
        monitors=[logging_hook])
    
        # Configure the accuracy metric for evaluation
    metrics = {
        "accuracy":
            learn.MetricSpec(
                metric_fn=tf.metrics.accuracy, prediction_key="classes"),
    }
    
    # Evaluate the model and print results
    eval_results = shapes_classifier.score(
        x=eval_data, y=eval_labels, metrics=metrics)
    print(eval_results)



    
    
    


In [76]:
try:
    tf.app.run()
except:
    print("Exception traceback:")
    print('-'*60)
    traceback.print_exc(file=sys.stdout)
    print('-'*60)

INFO:tensorflow:loading data...
INFO:tensorflow DONE
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12c5e0ad0>, '_model_dir': 'shapes_convnet_model', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into shapes_convnet_model/model.ckpt.
INFO:tensorflow:loss = 832.522, step = 1
INFO:tensorflow:probabilities = [[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0

In [55]:
#data, labels = random_shapes(32,2000)
#Binary data
np.save('rand_shapes_data_small.npy', data.astype(np.float32))
np.save('rand_shapes_labels_small.npy', labels.astype(np.int32))

In [56]:
data, labels = random_shapes(32, 500)
#Binary data
np.save('rand_shapes_data_test.npy', data.astype(np.float32))
np.save('rand_shapes_labels_test.npy', labels.astype(np.int32))

0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95


In [58]:
labels.dtype

dtype('int64')

In [66]:
a = add_channel(data)
a.shape

(500, 32, 32, 32, 1)